<a href="https://colab.research.google.com/github/alan-prudom/PyTiddlyWiki/blob/master/Obsidian_Vault_Project_and_Task_Scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import os
import datetime
import re

def scan_obsbian_vault(vault_path: str):
    """
    Scans the Obsidian vault for daily notes and extracts project/task information.
    Now supports daily notes in both BUJO/YYYY-MM-DD.md and BUJO/YYYY/YYYY-MM/YYYY-MM-DD.md conventions.
    File paths in the output are relative to the vault_path.
    Groups output by file path for clarity.

    Args:
        vault_path (str): The absolute path to your Obsidian vault.
    """
    print(f"Scanning Obsidian vault at: {vault_path}\n")

    bujo_base_folder = os.path.join(vault_path, 'BUJO')
    if not os.path.isdir(bujo_base_folder):
        print(f"Error: 'BUJO' base folder not found at {bujo_base_folder}. Please check the vault path and folder structure.")
        return

    today = datetime.date.today()
    missing_daily_notes = []
    # Store events as tuples: (display_path, line_num, line_content)
    planned_events = []
    events_needing_planning = []

    print("--- Daily Note Check (Last 10 Days) ---")
    for i in range(10):
        # Calculate the date for each of the last 10 days
        current_date = today - datetime.timedelta(days=i)

        # Format the date for filename (e.g., 2025-07-21.md)
        daily_note_filename = f"{current_date.strftime('%Y-%m-%d')}.md"

        # --- Check 1: Subfolder path (BUJO/YYYY/YYYY-MM/YYYY-MM-DD.md) ---
        year_folder = current_date.strftime('%Y')
        month_folder = current_date.strftime('%Y-%m')
        subfolder_daily_note_full_path = os.path.join(bujo_base_folder, year_folder, month_folder, daily_note_filename)

        # --- Check 2: Direct BUJO folder path (BUJO/YYYY-MM-DD.md) ---
        direct_bujo_daily_note_full_path = os.path.join(bujo_base_folder, daily_note_filename)

        found_full_path = None
        if os.path.exists(subfolder_daily_note_full_path):
            found_full_path = subfolder_daily_note_full_path
        elif os.path.exists(direct_bujo_daily_note_full_path):
            found_full_path = direct_bujo_daily_note_full_path

        if found_full_path:
            print(f"  [DEBUG] found_full_path: {found_full_path}")
            # Convert to relative path for display
            display_path = os.path.relpath(found_full_path, vault_path)
            print(f"  [DEBUG] display_path after relpath: {display_path}")

            # Explicitly remove the vault_path prefix if it exists
            if display_path.startswith(vault_path):
                display_path = display_path[len(vault_path):]
                print(f"  [DEBUG] display_path after removing vault_path: {display_path}")

            # Remove leading path separators if any
            if display_path.startswith(os.path.sep) or display_path.startswith(os.path.altsep):
                 display_path = display_path[1:]
                 print(f"  [DEBUG] display_path after removing leading separator: {display_path}")

            print(f"  [FOUND] Daily note for {current_date.strftime('%Y-%m-%d')} at {display_path}")
            # Process the daily note for events
            with open(found_full_path, 'r', encoding='utf-8') as f:
                for line_num, line in enumerate(f, 1):
                    stripped_line = line.strip()
                    # Check for planned events: lines starting with '- [<]'
                    if stripped_line.startswith('- [<]'):
                        # Check if it's an event needing planning: '- [<] Event'
                        if stripped_line.startswith('- [<] Event'):
                            events_needing_planning.append(
                                (display_path, line_num, stripped_line)
                            )
                        else:
                            # It's a general planned event
                            planned_events.append(
                                (display_path, line_num, stripped_line)
                            )
        else:
            # Convert expected paths to relative for display
            relative_subfolder_path = os.path.relpath(subfolder_daily_note_full_path, vault_path)
            if relative_subfolder_path.startswith(vault_path):
                relative_subfolder_path = relative_subfolder_path[len(vault_path):]
            if relative_subfolder_path.startswith(os.path.sep) or relative_subfolder_path.startswith(os.path.altsep):
                 relative_subfolder_path = relative_subfolder_path[1:]

            relative_direct_bujo_path = os.path.relpath(direct_bujo_daily_note_full_path, vault_path)
            if relative_direct_bujo_path.startswith(vault_path):
                relative_direct_bujo_path = relative_direct_bujo_path[len(vault_path):]
            if relative_direct_bujo_path.startswith(os.path.sep) or relative_direct_bujo_path.startswith(os.path.altsep):
                 relative_direct_bujo_path = relative_direct_bujo_path[1:]


            missing_daily_notes.append(
                f"Expected: {relative_subfolder_path} OR {relative_direct_bujo_path}"
            )
            print(f"  [MISSING] Daily note for {current_date.strftime('%Y-%m-%d')}")


    print("\n--- Summary Report ---")

    # Report missing daily notes
    if missing_daily_notes:
        print("\n!!! Action Required: Missing Daily Notes !!!")
        for expected_paths in missing_daily_notes:
            print(f"- {expected_paths}")
    else:
        print("\nAll daily notes for the last 10 days are present.")

    # Report events needing planning
    if events_needing_planning:
        print("\n!!! Action Required: Events Needing Planning !!!")
        print("The following events were flagged as needing further planning:")
        current_file_path = None
        for path, line_num, content in events_needing_planning:
            if path != current_file_path:
                print(f"--- File: {path} ---")
                current_file_path = path
            print(f"  - Line {line_num}: {content}")
    else:
        print("\nNo events flagged as needing planning.")

    # Report planned events
    if planned_events:
        print("\n--- Planned Events ---")
        print("The following planned events were found:")
        current_file_path = None
        for path, line_num, content in planned_events:
            if path != current_file_path:
                print(f"--- File: {path} ---")
                current_file_path = path
            print(f"  - Line {line_num}: {content}")
    else:
        print("\nNo planned events found.")

    print("\nScan complete.")


if __name__ == "__main__":
    # --- IMPORTANT: Configure your Obsidian vault path here ---
    # Replace this with the actual path to your Obsidian vault's root directory.
    # Example for Windows: 'C:\\Users\\YourUser\\Documents\\ObsidianVault'
    # Example for macOS/Linux: '/Users/YourUser/Documents/ObsidianVault'
    obsidian_vault_path = '/content/drive/MyDrive/3 Research/R-NotebookLM' # <<<--- CHANGE THIS LINE

    scan_obsidian_vault(obsidian_vault_path)

Scanning Obsidian vault at: /content/drive/MyDrive/3 Research/R-NotebookLM

--- Daily Note Check (Last 10 Days) ---
  [FOUND] Daily note for 2025-07-21 at /content/drive/MyDrive/3 Research/R-NotebookLM/BUJO/2025-07-21.md
  [FOUND] Daily note for 2025-07-20 at /content/drive/MyDrive/3 Research/R-NotebookLM/BUJO/2025/2025-07/2025-07-20.md
  [FOUND] Daily note for 2025-07-19 at /content/drive/MyDrive/3 Research/R-NotebookLM/BUJO/2025/2025-07/2025-07-19.md
  [FOUND] Daily note for 2025-07-18 at /content/drive/MyDrive/3 Research/R-NotebookLM/BUJO/2025/2025-07/2025-07-18.md
  [FOUND] Daily note for 2025-07-17 at /content/drive/MyDrive/3 Research/R-NotebookLM/BUJO/2025/2025-07/2025-07-17.md
  [FOUND] Daily note for 2025-07-16 at /content/drive/MyDrive/3 Research/R-NotebookLM/BUJO/2025/2025-07/2025-07-16.md
  [FOUND] Daily note for 2025-07-15 at /content/drive/MyDrive/3 Research/R-NotebookLM/BUJO/2025/2025-07/2025-07-15.md
  [FOUND] Daily note for 2025-07-14 at /content/drive/MyDrive/3 Researc

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
